<a href="https://colab.research.google.com/github/Gimmi-07/FFML_Projects_and_Labs/blob/main/FFML%20Module%2001%2CLAB%2002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT

FMML Module 01, Lab 02 <br>
 NAME: Grishma Yenchilwar ( grishma.yenchilwar.cse@ghrce.raisoni.net ) <br>
 Date: 11 Sep 2023 <br>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

**ANSWERS**

1)The percentage of the validation set in a machine learning workflow is a hyperparameter that can significantly impact the model's performance. Here's how increasing and reducing the percentage of the validation set can affect the accuracy of the model:

1. **Increasing the Percentage of Validation Set**:
   - **Pros**:
     - Increased Validation Data: With a larger validation set, the model gets more data to validate its performance. This can lead to a more reliable estimate of how well the model will generalize to unseen data.
     - Better Generalization: A larger validation set can help in detecting overfitting more effectively. It can prevent the model from fitting the training data noise.
   - **Cons**:
     - Smaller Training Set: As you allocate a larger percentage to the validation set, the training set size decreases. This can potentially reduce the model's ability to learn from the available data, especially if you have limited data to start with.
     - Longer Training Time: Training a model on a smaller training set may require more epochs or time to converge.

   Overall, increasing the percentage of the validation set can improve the accuracy estimation on validation data but may lead to a less optimized model due to the smaller training set.

2. **Reducing the Percentage of Validation Set**:
   - **Pros**:
     - Larger Training Set: A smaller validation set means a larger training set. This can allow the model to learn more from the training data and potentially lead to better performance.
     - Faster Training: With a larger training set, training times can be faster.
   - **Cons**:
     - Less Reliable Validation: A smaller validation set may provide a less reliable estimate of the model's generalization performance. The accuracy on the validation set may not be as indicative of the model's true performance on unseen data.
     - Risk of Overfitting: With less validation data, it may be harder to detect overfitting, and the model might generalize poorly to new data.

   Reducing the percentage of the validation set can lead to a model that trains faster and may perform better on the training data but might have a less accurate estimate of generalization performance.

In practice, the choice of the validation set size should be made based on factors such as the amount of available data, the complexity of the model, and the computational resources at hand. It's often a trade-off between reliable performance estimation and the ability to train a well-performing model. Techniques like cross-validation can also be used to mitigate some of the issues associated with small validation sets.

2)The size of the training and validation sets can significantly affect how well you can predict the accuracy on the test set using the validation set. This relationship is closely tied to the concept of **bias-variance trade-off** and generalization in machine learning.

Here's how the sizes of the training and validation sets can impact your ability to predict test set accuracy:

1. **Larger Training Set**:
   - **Pros**:
     - A larger training set provides more data for the model to learn from. This can lead to a better understanding of the underlying patterns in the data.
     - With a well-sized training set, the model is more likely to generalize well to unseen data, which includes the test set.

   - **Cons**:
     - If the validation set is relatively small compared to the training set, it may not provide a very accurate estimate of how well the model will perform on the test set.

2. **Larger Validation Set**:
   - **Pros**:
     - A larger validation set can provide a more reliable estimate of the model's generalization performance. It can reduce the variability in accuracy estimates.

   - **Cons**:
     - With a larger validation set, you allocate less data to the training set. This may lead to the model not learning as well from the available data.

3. **Balanced Sizes**:
   - Ideally, you want a balance between the sizes of the training and validation sets. A reasonable split is often 70-80% for training and 20-30% for validation.

4. **Overfitting and Underfitting**:
   - If the training set is too small and the model is too complex, it may overfit the training data, resulting in poor generalization. A larger validation set may help identify this issue.
   - If the training set is too large and the model is too simple, it may underfit the data, leading to a model that doesn't capture the underlying patterns. In this case, even a larger validation set might not help much.

In summary, the size of the training and validation sets should be chosen carefully based on various factors, including the amount of available data, the complexity of the model, and the need for accurate generalization estimates. The goal is to strike a balance between having enough data to train a good model and having a sufficient validation set to estimate performance accurately. Cross-validation techniques can also be used to mitigate the impact of the validation set size on performance estimation.

3)The percentage of data to reserve for the validation set depends on several factors, including the size of your overall dataset, the complexity of your model, and your specific goals for training and evaluation. There is no one-size-fits-all answer, but here are some general guidelines that can help you decide on a good percentage for the validation set:

1. **Typical Split**:
   - A common split is to reserve 70-80% of the data for training and allocate the remaining 20-30% for validation. This is a reasonable starting point for many datasets.

2. **Small Datasets**:
   - If you have a small dataset, you might need to allocate a larger percentage for validation to ensure that you have enough data to reliably estimate model performance. For example, you might use a 50-50 split or even a 60-40 split in such cases.

3. **Large Datasets**:
   - With a very large dataset, you can often afford to allocate a smaller percentage for validation because you still have a substantial amount of data for training. In such cases, a 90-10 or 95-5 split might be sufficient.

4. **Complex Models**:
   - If you are training a highly complex model with many parameters, you might need a larger validation set to detect overfitting. In such cases, consider allocating more data for validation.

5. **Simple Models**:
   - For simpler models that are less prone to overfitting, you might allocate a smaller percentage for validation.

6. **Cross-Validation**:
   - If you want a more robust estimate of model performance and are concerned about the choice of the validation set size, consider using techniques like k-fold cross-validation. This involves splitting the data into k subsets and performing k rounds of training and validation, rotating which subset is the validation set in each round.

7. **Domain Knowledge**:
   - Your domain knowledge and understanding of the problem can also guide your choice. Some problems may benefit from a larger validation set, while others may not require as much validation data.

In practice, you may need to experiment with different validation set sizes to find the one that works best for your specific task and dataset. The key is to ensure that the validation set is large enough to provide a reliable estimate of model performance while still leaving a sufficient amount of data for training.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**ANSWERS**

1)Yes, averaging the validation accuracy across multiple splits, such as in k-fold cross-validation, can provide more consistent and robust results compared to relying on a single validation split. This is one of the primary advantages of using cross-validation techniques in machine learning. Here's why it helps:

1. **Reduced Variance**: When you perform a single random split of your data into training and validation sets, the performance estimate can be highly dependent on the specific data points chosen for each set. This randomness can lead to significant variance in the accuracy estimate. By averaging the results over multiple splits, you reduce the impact of this variance.

2. **Better Generalization**: Cross-validation allows your model to be trained and evaluated on different subsets of the data multiple times. This helps ensure that your model's performance estimate is less sensitive to the idiosyncrasies of a single data split, and it provides a more representative estimate of how well your model is likely to perform on unseen data.

3. **Detecting Overfitting**: Cross-validation is particularly useful for detecting overfitting. If your model performs significantly better on the training data than on the validation data in multiple folds, it's a clear sign of overfitting. Averaging these results can provide a more reliable indication of overfitting.

4. **Model Selection**: In cases where you're comparing multiple models or hyperparameter settings, cross-validation allows you to assess their performance more comprehensively. You can select the model or settings that consistently perform well across different splits.

5. **More Data Used**: By repeatedly splitting the data and evaluating the model on different subsets, you effectively make use of a larger portion of your data for validation, which can lead to more accurate and stable performance estimates.

Common variations of cross-validation include k-fold cross-validation, stratified k-fold cross-validation, and leave-one-out cross-validation, among others. The choice of which variant to use depends on your specific dataset and objectives.

In summary, averaging validation accuracy across multiple splits through cross-validation is a valuable technique for obtaining more consistent and reliable performance estimates for your machine learning models. It is particularly helpful when you want to make informed decisions about model selection, hyperparameter tuning, and assessing the generalization capability of your models.

2)Averaging the validation accuracy across multiple splits using cross-validation does not directly give you a more accurate estimate of the test accuracy on unseen, out-of-sample data. The primary purpose of cross-validation is to provide a more reliable and robust estimate of how well your model is likely to generalize to new, unseen data, but it doesn't replace the need for a separate test set to obtain the actual test accuracy.

Here's how cross-validation and test sets work together:

1. **Cross-Validation**:
   - Cross-validation is used to estimate how well your model performs on the data it has been trained on. It involves splitting your dataset into multiple subsets (folds), training and validating your model on different combinations of these subsets, and then averaging the validation results. This process helps you assess how well your model is likely to generalize to similar data that it hasn't seen during training.
   - The cross-validation accuracy provides an estimate of how your model is expected to perform on new data drawn from the same distribution as your training data.

2. **Test Set**:
   - The test set is a separate and distinct dataset that your model has never seen during training or validation. It serves as a final evaluation step to obtain an accurate estimate of how well your model truly generalizes to unseen, out-of-sample data.
   - The test accuracy, which is calculated using the test set, provides a more accurate and unbiased estimate of your model's performance on real-world data.

In summary, while cross-validation helps you assess how well your model is likely to perform on similar data, it does not replace the need for a test set to obtain an accurate estimate of the model's performance on completely new and unseen data. Both cross-validation and a test set serve different purposes in evaluating and validating your machine learning model, and both are essential components of the model evaluation process.

3)The number of iterations or folds in cross-validation can have an impact on the estimate of model performance. The choice of the number of iterations depends on several factors, and it's important to strike a balance between computation time and the quality of the estimate. Here's how the number of iterations can affect the estimate:

1. **Fewer Iterations (e.g., k=3 or k=5)**:
   - Pros:
     - Faster Computation: Using fewer iterations is computationally less intensive, as you're training and validating the model on smaller subsets of the data.
     - Useful for Initial Assessment: In situations where you need a quick estimate of model performance, a lower number of iterations can provide a reasonable initial assessment.

   - Cons:
     - Higher Variance: With a smaller number of iterations, the estimate of model performance may have higher variance. It can be more sensitive to the specific data splits, leading to less stable results.

2. **More Iterations (e.g., k=10 or k=20)**:
   - Pros:
     - Lower Variance: Using more iterations reduces the variance in the estimate. You obtain a more stable and reliable assessment of model performance.

   - Cons:
     - Increased Computation Time: More iterations require training and validating the model multiple times, which can be computationally expensive, especially if your dataset is large or your model is complex.
     - Diminishing Returns: There is a point of diminishing returns, where increasing the number of iterations may not significantly improve the stability of the estimate but significantly increases computation time.

3. **Leave-One-Out Cross-Validation (LOOCV)**:
   - LOOCV is a special case of cross-validation where you use as many iterations as there are data points (i.e., k equals the number of data points).
   - Pros: LOOCV provides the least biased estimate of model performance because each data point serves as both a training and validation instance.
   - Cons: LOOCV is computationally expensive, especially for large datasets, and can be sensitive to noise in the data.

In general, the choice of the number of iterations (k) should be made considering the trade-off between computational resources and the need for a stable performance estimate. It's often recommended to start with a moderate value for k, such as 5 or 10, as this often strikes a reasonable balance between accuracy and computational efficiency. You can then adjust the value of k based on the specific characteristics of your dataset and the available computational resources.

4)Increasing the number of iterations in cross-validation can help mitigate some of the limitations associated with very small training or validation datasets to some extent. However, it does not completely replace the need for a reasonably sized dataset in each fold of cross-validation. Here's how increasing iterations can address the issue:

**Advantages of Increasing Iterations with Small Datasets:**

1. **Better Data Utilization**: With more iterations, you make better use of the limited data available. Each fold of cross-validation uses a different subset of the data for validation, which helps ensure that all data points are used for both training and validation.

2. **More Stable Estimates**: Increasing iterations can lead to more stable and reliable estimates of model performance. It reduces the impact of the specific data split on the performance estimate.

3. **Detecting Overfitting**: Cross-validation with more iterations can be effective in detecting overfitting even with small datasets. If the model consistently performs much better on the training data than on the validation data across multiple folds, it suggests overfitting.

**Limitations and Considerations:**

1. **Minimum Dataset Size**: Even with more iterations, there's a practical lower limit to how small your training and validation datasets can be. If these datasets are extremely tiny, the model may not be able to learn meaningful patterns or generalize effectively.

2. **Computation Time**: Increasing the number of iterations comes at the cost of increased computation time, especially if you have a computationally expensive model or a large dataset.

3. **Balance**: While increasing iterations can help with stability, it's still important to strike a balance between the number of iterations and the size of each fold. If you have an extremely small dataset, it's generally more beneficial to allocate a larger percentage to the training set, even if it means fewer iterations.

4. **Cross-Validation Strategy**: The effectiveness of increasing iterations may also depend on the specific cross-validation strategy you're using (e.g., k-fold, stratified k-fold, leave-one-out). Some strategies may be more suitable for very small datasets than others.

In summary, increasing the number of iterations in cross-validation can be a useful strategy to work with small datasets, but it should be used judiciously and in conjunction with an appropriate balance between training and validation dataset sizes. The goal is to obtain the most accurate and stable estimate of model performance while ensuring that the model has a reasonable amount of data to learn from during each iteration.